In [ ]:
import json
import time
import datetime
from kafka import KafkaConsumer
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate() 
SparkConf().getAll()

In [ ]:
def epoch_converter(datetime_obj):
    teste = datetime.datetime.strptime(datetime_obj, '%Y-%m-%d %H:%M:%S')
    epoch = datetime.datetime.timestamp(teste)
    return epoch

In [ ]:
consumer = KafkaConsumer('apelie_data')
home_page_messages = []
store_messages = []
product_messages = []
cart_messages = []
for msg in consumer:
    kafka_messages = json.loads(msg.value.decode('utf-8'))    
    if next(iter(kafka_messages)) == 'home_page':
        epoch = epoch_converter(kafka_messages['home_page']['check_in_timestamp']) 
        kafka_messages['home_page']['check_in_timestamp'] = int(epoch)
        epoch = epoch_converter(kafka_messages['home_page']['check_out_timestamp'])
        kafka_messages['home_page']['check_out_timestamp'] = int(epoch)
        home_page_messages.append(kafka_messages['home_page'])
        df_home = spark.createDataFrame(home_page_messages)
        df_home = df_home.withColumn('time_in_home_page', ( df_home['check_out_timestamp'] - df_home['check_in_timestamp'] ))
        df_home.show()
    elif next(iter(kafka_messages)) == 'store':
        epoch = epoch_converter(kafka_messages['store']['hour_of_checkin_store'])
        kafka_messages['store']['hour_of_checkin_store'] = int(epoch)
        epoch = epoch_converter(kafka_messages['store']['hour_of_checkout_sotre'])
        kafka_messages['store']['hour_of_checkout_sotre'] = int(epoch)
        store_messages.append(kafka_messages['store'])
        df_store = spark.createDataFrame(store_messages)
        df_store = df_store.withColumn('time_in_store', ( df_store['hour_of_checkout_sotre'] - df_store['hour_of_checkin_store'] ))
        df_store.show()
    elif next(iter(kafka_messages)) == 'product':
        epoch = epoch_converter(kafka_messages['product']['product_checkin'])
        kafka_messages['product']['product_checkin'] = int(epoch)
        epoch = epoch_converter(kafka_messages['product']['product_checkout'])
        kafka_messages['product']['product_checkout'] = int(epoch)
        product_messages.append(kafka_messages['product'])
        df_product = spark.createDataFrame(product_messages)
        df_product = df_product.withColumn('time_looking_product', ( df_product['product_checkout'] - df_product['product_checkin'] ))
        df_product.show()
    elif next(iter(kafka_messages)) == 'cart':
        cart_messages.append(kafka_messages['cart'])
        df_cart = spark.createDataFrame(cart_messages)
        df_cart.show()